In [2]:
import pandas as pd
from river import feature_extraction, linear_model, compose
from river import preprocessing
import difflib

In [3]:
# ==========================
# 1. Carregar o dataset
# ==========================
df = pd.read_excel("movies_database.xlsx", engine='openpyxl')


In [4]:
df.dtypes

id                        object
title                     object
link                      object
year                     float64
duration                  object
rating_mpa                object
rating_imdb              float64
vote                     float64
budget                   float64
gross_world_wide         float64
gross_us_canada          float64
gross_opening_weekend    float64
director                  object
writer                    object
star                      object
genre                     object
country_origin            object
filming_location          object
production_company        object
language                  object
win                      float64
nomination               float64
oscar                    float64
dtype: object

In [5]:
df['rating_imdb'] = df['rating_imdb'].astype(float)
df['year'] = df['year'].fillna(0).astype(int)
df['gross_world_wide'] = df['gross_world_wide'].astype(float)
df['budget'] = df['budget'].astype(float)
df['gross_opening_weekend'] = df['gross_opening_weekend'].astype(float)
df['oscar'] = df['oscar'].fillna(0).astype(int)
df['nomination'] = df['nomination'].fillna(0).astype(int)
df['duration'] = df['duration'].astype(str)

In [6]:
# Função para converter o tempo em minutos
def converter_tempo_para_minutos(tempo_str):
    if pd.isna(tempo_str) or tempo_str.lower() == 'nan':
        return None
    partes = tempo_str.replace('h', '').replace('m', '').split()
    try:
        horas = int(partes[0]) if partes[0] else 0
    except ValueError:
        horas = 0
    try:
        minutos = int(partes[1]) if len(partes) > 1 else 0
    except (ValueError, IndexError):
        minutos = 0
    return (horas * 60) + minutos

# Aplique a função à coluna 'duration' e crie uma nova coluna 'duration_min'
df['duration_min'] = df['duration'].apply(converter_tempo_para_minutos)

In [7]:
# ==========================
# 2. Definir intents e exemplos de perguntas
# ==========================
perguntas = [
    "Qual o filme mais bem avaliado?",
    "Qual filme tem a melhor nota?",
    "Me diga o filme mais bem avaliado.",
    "Qual o filme com maior bilheteria mundial?",
    "Qual filme arrecadou mais dinheiro?",
    "Qual filme teve maior receita?",
    "Quem é o diretor de Titanic?",
    "Quem dirigiu Titanic?",
    "Qual o nome do diretor do filme Titanic?",
    "Quantos Oscars ganhou Avatar?",
    "Quantos prêmios Oscar o filme Avatar recebeu?",
    "Avatar ganhou algum Oscar?",
    "Qual o gênero do filme Interstellar?",
    "Me diga o gênero de Interstellar.",
    "Interstellar é de qual gênero?",
    "Qual o filme com mais indicações ao Oscar?",
    "Qual filme recebeu mais indicações ao Oscar?",
    "Filme com maior número de indicações ao Oscar?",
    "Qual o filme mais longo?",
    "Qual é o filme com maior duração?",
    "Me diga o filme mais longo da lista.",
    "Qual o filme mais recente?",
    "Qual filme foi lançado mais recentemente?",
    "Me diga o filme mais novo."
 ]
intencoes = [
    "melhor_nota",
    "melhor_nota",
    "melhor_nota",
    "maior_bilheteria",
    "maior_bilheteria",
    "maior_bilheteria",
    "diretor",
    "diretor",
    "diretor",
    "oscar",
    "oscar",
    "oscar",
    "genero",
    "genero",
    "genero",
    "mais_indicacoes",
    "mais_indicacoes",
    "mais_indicacoes",
    "maior_duracao",
    "maior_duracao",
    "maior_duracao",
    "filme_recente",
    "filme_recente",
    "filme_recente"
 ]

In [61]:
# ==========================
# 3. Criar modelo com River
# ==========================
# Criação do modelo de classificação de intenções usando River
modelo = compose.Pipeline(
    ("bow", feature_extraction.BagOfWords(on="pergunta", lowercase=True, strip_accents=True, ngram_range=(1, 2))),
    ("clf", linear_model.LogisticRegression())
)


In [62]:
for pergunta, intent in zip(perguntas, intencoes):
    X = {"pergunta": pergunta}
    modelo = modelo.learn_one(X, intent)

ValueError: invalid literal for int() with base 10: 'melhor_nota'

In [ ]:
# Codificar as intenções como números usando um dicionário
# intent_to_num = {intent: idx for idx, intent in enumerate(intencoes)}

In [ ]:
# # Treinar com perguntas e intenções iniciais
# for pergunta, intent in zip(perguntas, intencoes):
#     # intent_encoded = intent_to_num[intent]
#     modelo.learn_one(pergunta, intent_encoded)
#     print(f"Pergunta: {pergunta} | Intenção: {intent} | Intenção codificada: {intent_encoded}")

Pergunta: Qual o filme mais bem avaliado? | Intenção: melhor_nota | Intenção codificada: 2
Pergunta: Qual filme tem a melhor nota? | Intenção: melhor_nota | Intenção codificada: 2
Pergunta: Me diga o filme mais bem avaliado. | Intenção: melhor_nota | Intenção codificada: 2
Pergunta: Qual o filme com maior bilheteria mundial? | Intenção: maior_bilheteria | Intenção codificada: 5
Pergunta: Qual filme arrecadou mais dinheiro? | Intenção: maior_bilheteria | Intenção codificada: 5
Pergunta: Qual filme teve maior receita? | Intenção: maior_bilheteria | Intenção codificada: 5
Pergunta: Quem é o diretor de Titanic? | Intenção: diretor | Intenção codificada: 8
Pergunta: Quem dirigiu Titanic? | Intenção: diretor | Intenção codificada: 8
Pergunta: Qual o nome do diretor do filme Titanic? | Intenção: diretor | Intenção codificada: 8
Pergunta: Quantos Oscars ganhou Avatar? | Intenção: oscar | Intenção codificada: 11
Pergunta: Quantos prêmios Oscar o filme Avatar recebeu? | Intenção: oscar | Intençã

In [32]:
# ==========================
# 4. Funções auxiliares
# ==========================
def encontrar_filme(titulo):
    """Retorna o filme mais parecido com o nome fornecido"""
    titulos = df['title'].tolist()
    candidato = difflib.get_close_matches(titulo, titulos, n=1, cutoff=0.6)
    return candidato[0] if candidato else None

In [33]:
num_to_intent = {v: k for k, v in intent_to_num.items()}

In [ ]:
def responder(pergunta):
    print(pergunta)
    # Predizer intenção
    # intent = modelo.predict([pergunta])[0]
    intent_num = modelo.predict_one(pergunta)
    intent = num_to_intent.get(intent_num, None)
    print(f"Pergunta: {pergunta} | Intenção: {intent}")
    
    # Respostas baseadas na intenção
    if intent == "melhor_nota":
        filme = df.loc[df['rating_imdb'].idxmax()]
        return f"O filme mais bem avaliado é '{filme['title']}' com nota {filme['rating_imdb']} no IMDb."
    
    elif intent == "maior_bilheteria":
        filme = df.loc[df['gross_world_wide'].idxmax()]
        return f"O filme com maior bilheteria mundial é '{filme['title']}' com ${filme['gross_world_wide']:,} arrecadados."
    
    elif intent == "diretor":
        palavras = pergunta.split()
        for p in palavras:
            filme = encontrar_filme(p)
            if filme:
                diretor = df.loc[df['title'] == filme, 'director'].values[0]
                return f"O diretor de '{filme}' é {diretor}."
        return "Não encontrei o filme na base de dados."
    
    elif intent == "oscar":
        palavras = pergunta.split()
        for p in palavras:
            filme = encontrar_filme(p)
            if filme:
                oscars = df.loc[df['title'] == filme, 'oscar'].values[0]
                return f"O filme '{filme}' ganhou {oscars} Oscars."
        return "Não encontrei informações sobre Oscars para esse filme."
    
    elif intent == "genero":
        palavras = pergunta.split()
        for p in palavras:
            filme = encontrar_filme(p)
            if filme:
                genero = df.loc[df['title'] == filme, 'genre'].values[0]
                return f"O gênero do filme '{filme}' é {genero}."
        return "Não encontrei o gênero do filme."
    
    elif intent == "mais_indicacoes":
        filme = df.loc[df['nomination'].idxmax()]
        return f"O filme com mais indicações ao Oscar é '{filme['title']}' com {filme['nomination']} indicações."
    
    elif intent == 20:
        filme = df.loc[df['duration_min'].idxmax()]
        return f"O filme mais longo é '{filme['title']}' com duração de {filme['duration_min']} minutos."
    
    elif intent == "filme_recente":
        filme = df.loc[df['year'].idxmax()]
        return f"O filme mais recente é '{filme['title']}', lançado em {filme['year']}."
    
    return "Desculpe, não consegui entender a sua pergunta."

In [59]:
# ==========================
# 5. Loop de interação
# ==========================
print("Bem-vindo ao sistema de filmes! Pergunte sobre os filmes ou digite 'sair' para encerrar.")

while True:
    pergunta_usuario = input("\nSua pergunta: ")

    if pergunta_usuario.lower() == "sair":
        print("Saindo...")
        break

    resposta = responder(pergunta_usuario)
    print(resposta)


Bem-vindo ao sistema de filmes! Pergunte sobre os filmes ou digite 'sair' para encerrar.
qual o filme mais longo?


AttributeError: 'Pipeline' object has no attribute 'predict'